In [1]:
import pandas as pd

In [2]:
non_compliance = pd.read_csv('../csv/dfps-2018-02-01/non_compliance.csv').drop('Unnamed: 0', axis=1)
operations = pd.read_csv('../csv/dfps-2018-02-01/operations.csv').drop('Unnamed: 0', axis=1)
inspections = pd.read_csv('../csv/dfps-2018-02-01/assessment.csv').drop('Unnamed: 0', axis=1)

In [3]:
# 747.2315(5) - Blankets, sheets, etc.
# 747.2327 - Infants sleeping on backs.
# 744.1203(4) - Responsibilities of Caregivers- Supervision of Children

dat_violations = non_compliance[
    non_compliance.STANDARD_NUMBER_DESCRIPTION.fillna('').str.contains(
        '747.2315\(5\)'
    ) | 
    non_compliance.STANDARD_NUMBER_DESCRIPTION.fillna('').str.contains(
        '747.2327'
    ) |
    non_compliance.STANDARD_NUMBER_DESCRIPTION.fillna('').str.contains(
        '744.1203\(4\)', 
    )
].assign(
    vio_code = lambda x: x.STANDARD_NUMBER_DESCRIPTION.str.split(' - ').str.get(0).replace(
        '747.2327', '747.2327 - Infants sleeping on backs'
    ).replace(
        '744.1203(4)', '744.1203(4) - Responsibilities of Caregivers- Supervision of Children'
    ).replace(
        '747.2315(5)', '747.2315(5) - Blankets, sheets, etc.'
    )
).merge(
    operations,
    on='OPERATION_ID'
).merge(
    inspections.drop(
        'OPERATION_ID',
        axis=1
    ),
    on='ACTIVITY_ID'
).query(
    'OPERATION_TYPE != "Child Placing Agency" & OPERATION_TYPE != "General Residential Operation"'
).where(
    # Only take 2016-02-01 to 2018-02-01
    lambda x: pd.to_datetime(x.ACTIVITY_DATE) >= pd.to_datetime('2016-02-01')
)

In [4]:
pd.crosstab(
    dat_violations.vio_code,
    dat_violations.OPERATION_TYPE
).assign(
    total = lambda x: x.apply(sum, axis=1)
)
#.to_csv('../export/blankets_responsibilities.csv')

OPERATION_TYPE,Licensed Center,Licensed Child-Care Home,Registered Child-Care Home,total
vio_code,,,,
744.1203(4) - Responsibilities of Caregivers- Supervision of Children,144,0,0,144
"747.2315(5) - Blankets, sheets, etc.",0,53,83,136
747.2327 - Infants sleeping on backs,0,11,10,21
